# Rebecca Godard

## Research question/interests

I am interested in using survival analysis or another analytic method to predict the length of time an incarcerated patient spends in psychiatric treatment. For example, I could use dignosis, type of crime/arrest, and demographic factors as predictors. 